In [2]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
import matplotlib.cm as cm
pd.set_option('display.max_columns', None)
from scipy import special
from scipy import integrate
from scipy.integrate import ode
from scipy.optimize import brentq
pi = np.pi

import pdb
import matplotlib.pyplot as plt
import scipy.constants as ct

from scipy.integrate import odeint, solve_ivp
from scipy.special import jv #bessel function of the first kind
from astropy.cosmology import Planck15 as cosmo
from scipy.interpolate import interp1d

In [3]:
def inspiral_time_peters(a0,e0,m1,m2,af=0):
    """
    Computes the inspiral time, in Gyr, for a binary
    a0 in Au, and masses in solar masses
    
    if different af is given, computes the time from a0,e0
    to that af
    
    for af=0, just returns inspiral time
    for af!=0, returns (t_insp,af,ef)
    """
    coef = 6.086768e-11 #G^3 / c^5 in au, gigayear, solar mass units
    beta = (64./5.) * coef * m1 * m2 * (m1+m2)
    
    if e0 == 0:
        #print e0,a0
        if not af == 0:
            print("ERROR: doesn't work for circular binaries")
            return 0
        return a0**4 / (4*beta)
    
    c0 = a0 * (1.-e0**2.) * e0**(-12./19.) * (1.+(121./304.)*e0**2.)**(-870./2299.)
    
    if af == 0:
        eFinal = 0.
    else:
        r = ode(deda_peters)
        r.set_integrator('lsoda')
        r.set_initial_value(e0,a0)
        r.integrate(af)
        if not r.successful():
            print("ERROR, Integrator failed!")
        else:
            eFinal = r.y[0]      
    
    time_integrand = lambda e: e**(29./19.)*(1.+(121./304.)*e**2.)**(1181./2299.) / (1.-e**2.)**1.5
    integral,abserr = integrate.quad(time_integrand,eFinal,e0)
    
    if af==0:
        return integral * (12./19.) * c0**4. / beta
    else:
        return (integral * (12./19.) * c0**4. / beta), af, eFinal

In [15]:
def dE_dT(m1,m2,a,e):
    print(m1, m2, a)
    '''
    m1 and M2 in grams
    a in cm 
    returns dE/dT in units of erg/s
    '''
    G = 6.67259*10**(-8)# cm3 gram-1 s-2
    c =  2.99792458*10**10 #cm s-1
    print(G**4 / c**5)
    print((m1*m2)**2 * (m1 + m2))
    print(a**5)
    print((1 + (73/24) * e**2 + (37/96) * e**4)/(1 - e**2)**(7/2))
    
    return -(32/5) * ((G**4 * (m1)**2 * m2 * (m1 + m2) ) / (c**5 * a**5 * (1 - e**2)**(7/2))) * (1 + (73/24) * e**2 + (37/96) * e**4)
    # return -(32/5) * ((G**4 * (m1)**2 * m2**2 * (m1 + m2) ) / (c**5 * a**5 * (1 - e**2)**(7/2))) * (1 + (73/24) * e**2 + (37/96) * e**4)

# Hulse-Taylor Binary Pulsar (
# m1 = 1.4411 #Msun
# m2 = 1.3867#Msun
# m1 *= Msun #convert to grams
# m2 *= Msun 
# a = 1.9501 * Rsun
# e = 0.6171


# print(dE_dT(m1,m2,a,e) * m2) #erg/s

In [16]:
m1 = 56986
m2 = 36.1669

Msun = 1.989*10**33 #gram
m1 *= Msun #convert to grams
m2 *= Msun 

Rsun = 6.957e+10 # cm in one RSUN
a = 1403230* Rsun
e = 0.999148

dt_s = 11694699182400.0

peters = dE_dT(m1,m2,a,e) #in erg/s
print("In ergs/s de/dt = ", peters )
print("In Joules, dE = ", peters * dt_s * 10**(-7))
print("")

AU = 1.496 * 10**13
print(a/AU,e,m1/Msun,m2/Msun)
t_insp = inspiral_time_peters(a/AU,e,m1/Msun,m2/Msun)

print("Inspiral time in Gyr is ", t_insp)

1.1334515400000001e+38 7.1935964100000005e+34 9.76227111e+16
8.186047238099782e-82
7.540090365046894e+183
8.866542943532093e+84
21676263018.741184
In ergs/s de/dt =  -1.3425015411786425e-06
In Joules, dE =  -1.5700151675992609

6525.5822927807485 0.999148 56986.0 36.1669
Inspiral time in Gyr is  1721.2222357039486


In [11]:
m1 = 56986
m2 = 42

Msun = 1.989*10**33 #gram
m1 *= Msun #convert to grams
m2 *= Msun 

Rsun = 6.957e+10 # cm in one RSUN
a = 51760.1* Rsun
e = 0.70152

dt_s = 11694699182400.0

peters = dE_dT(m1,m2,a,e) #in erg/s
print("In ergs/s de/dt = ", peters )
print("In Joules, dE = ", peters * dt_s * 10**(-7))
print("")

AU = 1.496 * 10**13
print(a/AU,e,m1/Msun,m2/Msun)
t_insp = inspiral_time_peters(a/AU,e,m1/Msun,m2/Msun)

print("Inspiral time in Gyr is ", t_insp)

1.1334515400000001e+38 8.353800000000001e+34 3600950157000000.0
8.186047238099782e-82
1.016943874817336e+184
6.054601307706279e+77
12374418233.141401
In ergs/s de/dt =  -1.0889061363844197e+36
In Joules, dE =  -1.2734429702885216e+42

240.70522439839573 0.999 56986.0 42.0
Inspiral time in Gyr is  0.004782169336776545


In [17]:
#Converting back to code units 
l_conv = 1.23428e+19 #code unit of length in cgs
tn_conv = 1.37787e+12 #NBody time in cgs

dt = 8.48752
print(peters)
print('Per unit mass in CMC code units', peters * ((tn_conv)**3/(l_conv)**2))

-1.3425015411786425e-06
Per unit mass in CMC code units -2.305218746254162e-08


In [12]:
#Converting back to code units 
#First, make it per unit mass
Msun = 1.989*10**33 #gram

specificd_E_dT = dE_dT(m1,m2,a,e)/ (m2*Msun)
print(specificd_E_dT)
l_conv = 1.23428e+19
tn_conv = 1.37787e+12
#coverting to code units 
final_E = specificd_E_dT * (l_conv/tn_conv)**-2

print(final_E)


-1.7590165126113933e-76
-2.1920932341466267e-90


In [8]:
8.48752*1.37787e+12

11694699182400.0